In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import seaborn as sns

In [7]:
years = [2023]

for year in years:
    col1 = ["ISO Country Numeric Code", "ISO Country Code", "Country", "Region",
            "Survey", "Survey Year", "MPI", "Headcount", "Intensity", "Vulnerable to Poverty",
            "Severe Poverty", "Headcount Destitution", "MPI Poor Destitute", "Poor Variance",
            "Population YOS", f"Population {year - 3}", f"Population {year - 2}", "Poor YOS",
            f"Poor Population {year - 3}", f"Poor Population {year - 2}", "Total Indicators",
            "Indicator Missing"]


    file_path = f"../../data/raw/Global MPI {year} National Results.xlsx"

    df_1 = pd.read_excel("../../data/raw/Global MPI 2023 National Results.xlsx",
                     sheet_name= "1.1 National MPI Results",
                     skiprows=8, skipfooter=10, names = col1)
    df_2 = pd.read_excel("../../data/raw/Global MPI 2023 National Results.xlsx",
                     sheet_name= "1.3 Contribut'n of Deprivations",
                     skiprows=8, skipfooter=10)
    
    df = pd.merge

,ISO Country Numeric Code,ISO Country Code,Country,Region,Survey,Survey Year,MPI,Headcount,Intensity,Vulnerable to Poverty,...,MPI Poor Destitute,Poor Variance,Population YOS,Population 2020,Population 2021,Poor YOS,Poor Population 2020,Poor Population 2021,Total Indicators,Indicator Missing
0,688,SRB,Serbia,Europe and Central Asia,MICS,2019,0.000433,0.113674,38.101535,2.098999,...,0.000000,...*,7401.056,7358.005,7296.769,8.413053,8.364115,8.294505,10,NaN
1,51,ARM,Armenia,Europe and Central Asia,DHS,2015-2016,0.000690,0.190553,36.213947,2.776360,...,13.456203,...*,2865.835,2805.608,2790.974,5.460946,5.346181,5.318295,10,NaN
2,804,UKR,Ukraine,Europe and Central Asia,MICS,2012,0.000840,0.244237,34.410454,0.418627,...,62.432526,...*,45406.226,43909.666,43531.422,110.898956,107.243797,106.319984,9,Nutrition
3,795,TKM,Turkmenistan,Europe and Central Asia,MICS,2019,0.000849,0.249407,34.047836,0.341083,...,41.415451,...*,6158.420,6250.438,6341.855,15.359540,15.589040,15.817040,9,Cooking fuel
4,268,GEO,Georgia,Europe and Central Asia,MICS,2018,0.001245,0.340126,36.592345,2.083891,...,28.635067,...*,3772.325,3765.912,3757.980,12.830652,12.808841,12.781861,10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,450,MDG,Madagascar,Sub-Saharan Africa,DHS,2021,0.385927,68.418862,56.406582,15.371963,...,44.444210,0.025589,28915.653,28225.177,28915.653,19783.759766,19311.345703,19783.759766,10,NaN
106,108,BDI,Burundi,Sub-Saharan Africa,DHS,2016-2017,0.408861,75.097473,54.444055,15.761258,...,43.306606,0.021947,11155.593,12220.227,12551.213,8377.568359,9177.082031,9425.643555,10,NaN
107,140,CAF,Central African Republic,Sub-Saharan Africa,MICS,2018-2019,0.461348,80.414178,57.371416,12.902057,...,56.963650,0.025142,5209.324,5343.020,5457.154,4189.035156,4296.545410,4388.325684,10,NaN
108,148,TCD,Chad,Sub-Saharan Africa,MICS,2019,0.517011,84.174947,61.421031,10.665950,...,75.703453,0.024102,16126.866,16644.701,17179.740,13574.781250,14010.667969,14461.037109,10,NaN
